In [11]:
# Initialize the random number generator
rng = MersenneTwister(2016);

In [12]:
function pretty_print(v)
    @printf "["
    for i=1:length(v)
        @printf "%8.5f" v[i]
        if i < length(v)
            @printf "  "
        end
    end
    @printf "]\n"
end    

pretty_print (generic function with 1 method)

In [13]:
# Size of matrix
n = 4
Q = rand(rng, n, n)
Q, = qr(Q)

Λ = diagm(Float32[2.0^(-i) for i=0:n-1])

A = Q * Λ * Q'

4x4 Array{Float64,2}:
  0.530496   0.290802   0.221849  -0.0953488
  0.290802   0.446154   0.204698   0.0396639
  0.221849   0.204698   0.572017   0.143619 
 -0.0953488  0.0396639  0.143619   0.326334 

In [14]:
r = 2
Qk = rand(rng, n, r)

for k=1:4
    Qk = A * Qk
    Qk, = qr(Qk)
    for j=1:2
        if Qk[1,j] * Q[1,j] < 0
            Qk[:,j] = -Qk[:,j]
            # This is not necessary but avoids conflicting signs
        end
    end
    println("\nIteration ",k)
    pretty_print(Qk[:,1])
    pretty_print(Q[:,1])
    pretty_print(Qk[:,2])
    pretty_print(Q[:,2])
end


Iteration 1
[-0.45382  -0.40682  -0.71866  -0.33477]
[-0.59721  -0.53726  -0.59106  -0.07311]
[ 0.48450   0.62440  -0.48502  -0.37435]
[ 0.49054   0.13115  -0.53093  -0.67844]

Iteration 2
[-0.51027  -0.49695  -0.67445  -0.19436]
[-0.59721  -0.53726  -0.59106  -0.07311]
[ 0.51902   0.39708  -0.52932  -0.54108]
[ 0.49054   0.13115  -0.53093  -0.67844]

Iteration 3
[-0.55270  -0.52190  -0.63599  -0.13288]
[-0.59721  -0.53726  -0.59106  -0.07311]
[ 0.51426   0.26596  -0.53696  -0.61358]
[ 0.49054   0.13115  -0.53093  -0.67844]

Iteration 4
[-0.57508  -0.53058  -0.61413  -0.10301]
[-0.59721  -0.53726  -0.59106  -0.07311]
[ 0.50480   0.19864  -0.53580  -0.64703]
[ 0.49054   0.13115  -0.53093  -0.67844]


In [15]:
Qk = rand(rng, n, r)
niter = 30
err = zeros(niter,2)

for k=1:niter
    Qk = A * Qk
    Qk, Rk = qr(Qk)
    for j=1:2
        if Qk[1,j] * Q[1,j] < 0
            Qk[:,j] = -Qk[:,j]
        end
        err[k,j] = norm(Qk[:,j] - Q[:,j])
    end
end
err

30x2 Array{Float64,2}:
 0.258078     1.53948   
 0.126301     1.15456   
 0.0627601    0.917151  
 0.0313297    0.602246  
 0.0156585    0.335526  
 0.00782845   0.173239  
 0.00391413   0.0873537 
 0.00195705   0.0437703 
 0.000978524  0.0218969 
 0.000489262  0.0109499 
 0.000244631  0.00547514
 0.000122315  0.00273759
 6.11577e-5   0.0013688 
 ⋮                      
 9.55589e-7   2.13875e-5
 4.77794e-7   1.06937e-5
 2.38897e-7   5.34687e-6
 1.19449e-7   2.67344e-6
 5.97243e-8   1.33672e-6
 2.98622e-8   6.68359e-7
 1.49311e-8   3.34179e-7
 7.46554e-9   1.6709e-7 
 3.73277e-9   8.35449e-8
 1.86638e-9   4.17724e-8
 9.33192e-10  2.08862e-8
 4.66596e-10  1.04431e-8

In [16]:
using Plots
plotlyjs()

Plots.PlotlyJSBackend()

In [17]:
plot(err[:,1],lab="Error in q1",yscale=:log10,left_margin=30mm)
plot!(err[:,2],lab="Error in q2")
plot!(Float32[4.0^(-i) for i=1:niter],lab="Theoretical convergence",linestyle=:dash)
xaxis!("Iteration")
yaxis!("Error")